# Problema delle n regine - Simulated Annealing

## Importazione delle librerie Python e parametri

In [1]:
import random # Generazione dei numeri random
import numpy as np # Per la scacchiera (matrice)
import math # Per la probabilità

In [2]:
TEMP_I = 30 # T iniziale
TEMP_F = 0.2 # T finale
ALFA = 0.99 # prodotto per diminuire t
STEPS_PER_CHANGE = 100 # numero di step per ciclo

DIM = 20   # DIM della scacchiera (NXN)

## Scacchiera e regine

Tweak: per scambiare due colonne

In [3]:
def tweak(sol):
    sol_copy = np.copy(sol) # Copia la soluzione
    x = random.randint(0, DIM-1) # indice colonna 1
    y = random.randint(0, DIM-1) # indice colonna 2
    # Per evitare che le due colonne scelta siano uguali
    while x == y:
        y = random.randint(0, DIM-1)
    # Scambia le due colonne
    temp = sol_copy[y]
    sol_copy[y] = sol_copy[x]
    sol_copy[x] = temp
    return sol_copy

Inizializza lo stato inziale della scacchiera

In [4]:
def init(sol):
    # Esegui tante volte quante sono le colonne
    # della scacchiera
    for ind in range(0, DIM-1):
        sol = tweak(sol)
    return sol


Stampa della scacchiera

In [5]:
def print_board(sol):
    board = [[0] * DIM for i in range(DIM)]
    for x in range(0,DIM):
        board[sol[x]][x] = 'Q'
    print("SCACCHIERA", '\n')
    for x in range(0,DIM):
        for y in range(0,DIM):
            if(board[x][y]=='Q'):
                print("Q   ",end=''),
            else:
                print(".   ",end=''),
        print("\n")
    print("\n\n")

## Calcolo dell'energia

In [6]:
def energy(state):
    # Creazione della scacchiera
    board = [[0] * DIM for ind in range(DIM)]
    # Posizione delle regine
    for col in range(0, DIM):
        board[state[col]][col] = 'Q'
    # Possibili spostamenti sulla scacchiera
    dx = [-1, 1, -1, 1]
    dy = [-1, 1, 1, -1]
    # Calcolo del numero di conflitti
    conflitti = 0
    for ind in range(0, DIM):
        x = state[ind] # regina
        y = ind # colonna corrente
        # Per gli attacchi in diagonale
        for inx in range(0, 4):
            tempx = x # per spostarsi su x
            tempy = y # spostarsi su y
            while True:
                tempx += dx[inx] # sposta su x
                tempy += dy[inx] # sposta su y
                if  ((tempx < 0) or
                    (tempx >= DIM) or
                    (tempy < 0) or
                    (tempy >= DIM)):
                    break # hai raggiunto il bordo
                if board[tempx][tempy] == 'Q':
                    conflitti += 1 # trovata una regina
    return conflitti

## Algoritmo vero e proprio

In [7]:
def simulated_annealing():
    # Stato iniziale
    current = init(range(0, DIM)) # stato di current
    current_energy = energy(current) # valore di current
    # Inizializza la best
    best = current
    best_energy = current_energy
    temperature = TEMP_I
    # Esecuzione dell'algoritmo
    while temperature > TEMP_F and best_energy != 0:
        for step in range(0, STEPS_PER_CHANGE):
            succ = False # verifica se lo stato e' un successore
            next = tweak(current) # stato di next
            next_energy = energy(next) # valore di next
            if next_energy < current_energy:
                # Successore accettato
                succ = True
            else:
                # Calcolo probabilistico
                delta = next_energy - current_energy
                prop = math.exp(-delta/temperature) # link function
                test = random.random()
                if test < prop:
                    # Successore accettato
                    succ = True
            if succ:
                # Poni current uguale a next
                current = next
                current_energy = next_energy
                if current_energy < best_energy:
                    # Nuovo best trovato
                    best = current
                    best_energy = current_energy
        # Diminuisci la temperatura
        # N.B.: Se moltipli che 0.99 non arriverai mai a 0
        temperature *= ALFA
    return best

## Main

In [8]:
solution = simulated_annealing()

In [9]:
print_board(solution)

SCACCHIERA 

.   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   

.   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   

.   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   

.   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

Q  